In [1]:
import os

In [2]:
%pwd

'c:\\Users\\VAIO\\OneDrive\\Desktop\\Chiken_disease_classification_project\\Chiken_disease_classification\\research'

In [3]:
%cd ..

c:\Users\VAIO\OneDrive\Desktop\Chiken_disease_classification_project\Chiken_disease_classification


In [4]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is E0B8-81FC

 Directory of c:\Users\VAIO\OneDrive\Desktop\Chiken_disease_classification_project\Chiken_disease_classification

03/17/2025  11:10 AM    <DIR>          .
03/17/2025  11:10 AM    <DIR>          ..
03/10/2025  08:38 AM    <DIR>          .github
03/17/2025  11:13 AM             3,599 .gitignore
03/17/2025  11:11 AM    <DIR>          artifacts
03/13/2025  10:38 AM    <DIR>          config
03/10/2025  09:50 AM                 0 dvc.yaml
03/13/2025  09:03 AM    <DIR>          logs
03/17/2025  11:11 AM               459 main.py
03/17/2025  11:32 AM               160 params.yaml
03/13/2025  10:38 AM               276 README.md
03/12/2025  09:10 AM               154 requirements.txt
03/17/2025  11:17 AM    <DIR>          research
03/18/2025  10:10 AM               824 setup.py
03/10/2025  08:38 AM    <DIR>          src
03/10/2025  09:50 AM             1,325 template.py
03/10/2025  09:51 AM    <DIR>          templates
        

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path:Path
    updated_base_model_path: Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weights:str
    params_classes:int
    

In [27]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self):
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        return prepare_base_model_config

        

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2771220601.py, line 16)

In [29]:
import os
import urllib.request as request
from zipfile import ZipFile
# import tensorflow as tf



In [30]:
# class PrepareBaseModel:
#     def __init__(self,config:PrepareBaseModelConfig):
#         self.config = config
    
#     def get_base_model(self):
#         self.model = tf.keras.applications.vgg16.VGG16(
#             input_shape = self.config.params_weights,
#             include_top = self.config.params_include_top
#         )
#         self.save_model(path=self.config.base_model_path,model=self.model)
    
#     @staticmethod
#     def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
#         if freeze_all:
#             for layer in model.layers:
#                 model.trainable = False
#         elif (freeze_till is not None) and (freeze_till>0):
#             for layer in model.layers[:freeze_till]:
#                 model.trainable = False
            
#         flatten_in = tf.keras.layers.Flatten()(model.output)
#         prediction = tf.keras.layers.Dense(
#             units=classes,
#             activation = "softmax"
#         )(flatten_in)
#         full_model = tf.keras.models.Model(
#             input = model.input,
#             outputs = prediction
#         )
#         full_model.compile(
#             optimizer = tf.keras.optimizer.SGD(learning_rate=learning_rate),
#             loss = tf.keras.losses.CategoricalCrossentropy(),
#             metrics = ["accuracy"]
#         )
#         full_model.summary()
#         return full_model
        
#     def update_base_model(self):
#         self.full_model = self._prepare_full_model(
#             model = self.model,
#             classes=self.config.params_classes,
#             freeze_all=True,
#             freeze_till=None,
#             learning_rate=self.config.params_learning_rate
#         )
#         self.save_model(path=self.config.updated_base_model_path,model=self.full_model)
    
#     @staticmethod
#     def save_model(path:Path,model:tf.keras.Model):
#         model.save(path)

In [31]:
# The above code using pytorch
import torch
import torch.nn as nn
import torchvision.models as models 
from pathlib import Path

In [32]:
 

class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = models.vgg16(pretrained=True if self.config.params_weights=="imagenet" else False)
        if not self.config.params_include_top:
            self.model = nn.Sequential(*list(self.model.children())[:-1])
        self.save_model(path=self.config.base_model_path,model=self.model)
    
    @staticmethod
    def _prepare_full_model(model,classes,learning_rate):
        # Freeze all parameters
        for param in model.parameters():
            param.requires_grad = False
        num_features = model.classifier[0].in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features,classes),
            nn.Softmax(dim=1)
        )
        optimizer = torch.optim.SGD(model.parameters,lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        return model , optimizer,criterion
    
    def update_base_model(self):
        self.Full_model,optimizer,criterion = self._prepare_full_model(model=self.model,
                                                                       classes=self.config.params_classes,learning_rate=self.config.params_learning_rate)
        self.save_model(path=self.config.updated_base_model_path,model=self.Full_model)
    
    @staticmethod
    def save_model(path:Path,model):
        torch.save(model.state_dict(),path)
         
    


In [33]:
try:
    config = ConfigurationManager()
    PrepareBaseModelConfig = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=PrepareBaseModelConfig)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e

[2025-03-20 09:21:47,520:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-03-20 09:21:47,586:INFO:common:yaml file: params.yaml loaded successfully]
[2025-03-20 09:21:47,601:INFO:common:created directory at : artifacts]
[2025-03-20 09:21:47,615:INFO:common:created directory at : artifacts/prepare_base_model]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\VAIO\OneDrive\Desktop\Chiken_disease_classification_project\chicken_disease_classification_env\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\VAIO\AppData\Local\Temp\ipykernel_13456\1279191077.py", line 9, in <module>
    raise e
  File "C:\Users\VAIO\AppData\Local\Temp\ipykernel_13456\1279191077.py", line 3, in <module>
    PrepareBaseModelConfig = config.get_prepare_base_model_config()
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\VAIO\AppData\Local\Temp\ipykernel_13456\1775423162.py", line 13, in get_prepare_base_model_config
    prepare_base_model_config = PrepareBaseModelConfig(
                                ^^^^^^^^^^^^^^^^^^^^^^^
TypeError: PrepareBaseModelConfig.__init__() missing 1 required positional argument: 'updated_base_model_path'

During handling of the above exception, another ex